In [41]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# THIẾT LẬP MÔI TRƯỜNG VÀ TỪI DỮ LIỆU

In [42]:
df_train = pd.read_csv('hwu/train.csv', sep=',', header=None, names=['text', 'intent'])
df_val = pd.read_csv('hwu/val.csv', sep=',', header=None, names=['text', 'intent'])
df_test = pd.read_csv('hwu/test.csv', sep=',', header=None, names=['text', 'intent'])
print(f"Train shape: {df_train.shape}")
print(f"Validation shape: {df_val.shape}")
print(f"Test shape: {df_test.shape}")
print(f"\nSố lượng intent: {df_train['intent'].nunique()}")

Train shape: (8955, 2)
Validation shape: (1077, 2)
Test shape: (1077, 2)

Số lượng intent: 65


# Tiền xử lý nhãn

In [43]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(pd.concat([df_train['intent'], df_val['intent'], df_test['intent']]))

y_train = label_encoder.transform(df_train['intent'])
y_val = label_encoder.transform(df_val['intent'])
y_test = label_encoder.transform(df_test['intent'])

num_classes = len(label_encoder.classes_)
print(f"\nSố lượng classes: {num_classes}")


Số lượng classes: 65


# TF-IDF + LOGISTIC REGRESSION

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, f1_score

# Tạo pipeline
tfidf_lr_pipeline = make_pipeline(
    TfidfVectorizer(max_features=5000, lowercase=True),
    LogisticRegression(max_iter=1000, random_state=42)
)

# Huấn luyện
print("Đang huấn luyện TF-IDF + Logistic Regression...")
tfidf_lr_pipeline.fit(df_train['text'], y_train)

# Đánh giá
y_pred_tfidf = tfidf_lr_pipeline.predict(df_test['text'])
f1_tfidf = f1_score(y_test, y_pred_tfidf, average='macro')

print(f"\nF1-score (Macro) trên test set: {f1_tfidf:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_tfidf, 
                          target_names=label_encoder.classes_, 
                          digits=4))

Đang huấn luyện TF-IDF + Logistic Regression...

F1-score (Macro) trên test set: 0.8226

Classification Report:
                          precision    recall  f1-score   support

             alarm_query     0.9000    0.9474    0.9231        19
            alarm_remove     1.0000    0.7273    0.8421        11
               alarm_set     0.8095    0.8947    0.8500        19
       audio_volume_down     1.0000    0.7500    0.8571         8
       audio_volume_mute     0.9231    0.8000    0.8571        15
         audio_volume_up     0.9286    1.0000    0.9630        13
          calendar_query     0.4762    0.5263    0.5000        19
         calendar_remove     0.8947    0.8947    0.8947        19
            calendar_set     0.8235    0.7368    0.7778        19
                category     0.0000    0.0000    0.0000         1
          cooking_recipe     0.5909    0.6842    0.6341        19
        datetime_convert     0.6667    0.7500    0.7059         8
          datetime_query     

# WORD2VEC (AVERAGE) + DENSE LAYER

In [45]:

from gensim.models import Word2Vec
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

# 1. Huấn luyện Word2Vec
print("Đang huấn luyện Word2Vec...")
sentences_train = [text.lower().split() for text in df_train['text']]
w2v_model = Word2Vec(sentences=sentences_train, 
                     vector_size=100, 
                     window=5, 
                     min_count=1, 
                     workers=4,
                     seed=42)

print(f"Vocab size: {len(w2v_model.wv)}")

# 2. Hàm chuyển câu thành vector trung bình
def sentence_to_avg_vector(text, model):
    """Chuyển câu thành vector trung bình của các từ"""
    words = text.lower().split()
    vectors = []
    for word in words:
        if word in model.wv:
            vectors.append(model.wv[word])
    
    if len(vectors) == 0:
        return np.zeros(model.vector_size)
    
    return np.mean(vectors, axis=0)

# 3. Tạo dữ liệu train/val/test
print("Đang tạo vector trung bình cho các câu...")
X_train_avg = np.array([sentence_to_avg_vector(text, w2v_model) 
                        for text in df_train['text']])
X_val_avg = np.array([sentence_to_avg_vector(text, w2v_model) 
                      for text in df_val['text']])
X_test_avg = np.array([sentence_to_avg_vector(text, w2v_model) 
                       for text in df_test['text']])

# One-hot encoding cho labels
y_train_cat = to_categorical(y_train, num_classes)
y_val_cat = to_categorical(y_val, num_classes)
y_test_cat = to_categorical(y_test, num_classes)

# 4. Xây dựng mô hình Dense
print("Đang xây dựng mô hình Dense...")
w2v_model_dense = Sequential([
    Dense(128, activation='relu', input_shape=(w2v_model.vector_size,)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

w2v_model_dense.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print(w2v_model_dense.summary())

# 5. Huấn luyện
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

print("Đang huấn luyện mô hình...")
history_w2v = w2v_model_dense.fit(
    X_train_avg, y_train_cat,
    validation_data=(X_val_avg, y_val_cat),
    epochs=50,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

# 6. Đánh giá
test_loss_w2v, test_acc_w2v = w2v_model_dense.evaluate(X_test_avg, y_test_cat, verbose=0)
y_pred_w2v = np.argmax(w2v_model_dense.predict(X_test_avg, verbose=0), axis=1)
f1_w2v = f1_score(y_test, y_pred_w2v, average='macro')

print(f"\nTest Loss: {test_loss_w2v:.4f}")
print(f"Test Accuracy: {test_acc_w2v:.4f}")
print(f"F1-score (Macro): {f1_w2v:.4f}")


Đang huấn luyện Word2Vec...
Vocab size: 4424
Đang tạo vector trung bình cho các câu...
Đang xây dựng mô hình Dense...


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_7 (Dense)                 │ (None, 128)            │        12,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 65)             │         4,225 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,409 (99.25 KB)

 Trainable params: 25,409 (99.25 KB)

 Non-trainable params: 0 (0.00 B)

None
Đang huấn luyện mô hình...
Epoch 1/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0171 - loss: 4.1700 - val_accuracy: 0.0297 - val_loss: 4.1141
Epoch 2/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0293 - loss: 4.1094 - val_accuracy: 0.0622 - val_loss: 3.9886
Epoch 3/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0544 - loss: 3.9588 - val_accuracy: 0.0743 - val_loss: 3.7368
Epoch 4/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0672 - loss: 3.7519 - val_accuracy: 0.0947 - val_loss: 3.6062
Epoch 5/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0742 - loss: 3.6796 - val_accuracy: 0.0938 - val_loss: 3.5645
Epoch 6/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0827 - loss: 3.6072 - val_accuracy: 0.1049 - val_loss: 3.5214
Epoch 7/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0850 - loss: 3.5847 - val_accuracy: 0.1198 - val_loss: 3.4384
Epoch 8/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0995 -

# EMBEDDING (SCRATCH) + LSTM

In [46]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 1. Tạo tokenizer
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(df_train['text'])

# 2. Lấy vocab size
vocab_size_actual = len(tokenizer.word_index) + 1
print("Vocab size thực tế:", vocab_size_actual)

# 3. Chuyển văn bản thành sequence
X_train_seq = tokenizer.texts_to_sequences(df_train['text'])
X_val_seq = tokenizer.texts_to_sequences(df_val['text'])
X_test_seq = tokenizer.texts_to_sequences(df_test['text'])

# 4. Padding
max_len = 100   # hoặc tùy dataset
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_val_pad = pad_sequences(X_val_seq, maxlen=max_len, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post')


Vocab size thực tế: 4265


In [47]:
# Lấy kích thước vocab thực tế
vocab_size_actual = len(tokenizer.word_index) + 1
print("Vocab size thực tế:", vocab_size_actual)

# Số lớp
num_classes = y_train_cat.shape[1]

# Xây dựng mô hình
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

print("Đang xây dựng mô hình LSTM (Scratch)...")
lstm_model_scratch = Sequential([
    Embedding(
        input_dim=vocab_size_actual,
        output_dim=100,
        input_length=max_len
    ),
    LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

lstm_model_scratch.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print(lstm_model_scratch.summary())

# Huấn luyện
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

print("Đang huấn luyện mô hình LSTM (Scratch)...")
history_lstm_scratch = lstm_model_scratch.fit(
    X_train_pad, y_train_cat,
    validation_data=(X_val_pad, y_val_cat),
    epochs=50,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

# Đánh giá
test_loss_lstm_scratch, test_acc_lstm_scratch = lstm_model_scratch.evaluate(
    X_test_pad, y_test_cat, verbose=0)
y_pred_lstm_scratch = np.argmax(lstm_model_scratch.predict(X_test_pad, verbose=0), axis=1)
f1_lstm_scratch = f1_score(y_test, y_pred_lstm_scratch, average='macro')

print(f"\nTest Loss: {test_loss_lstm_scratch:.4f}")
print(f"Test Accuracy: {test_acc_lstm_scratch:.4f}")
print(f"F1-score (Macro): {f1_lstm_scratch:.4f}")


Vocab size thực tế: 4265
Đang xây dựng mô hình LSTM (Scratch)...


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None
Đang huấn luyện mô hình LSTM (Scratch)...
Epoch 1/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 16s 49ms/step - accuracy: 0.0209 - loss: 4.1588 - val_accuracy: 0.0176 - val_loss: 4.1374
Epoch 2/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 14s 51ms/step - accuracy: 0.0139 - loss: 4.1391 - val_accuracy: 0.0176 - val_loss: 4.1324
Epoch 3/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 14s 49ms/step - accuracy: 0.0169 - loss: 4.1362 - val_accuracy: 0.0176 - val_loss: 4.1302
Epoch 4/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 14s 51ms/step - accuracy: 0.0185 - loss: 4.1325 - val_accuracy: 0.0176 - val_loss: 4.1324
Epoch 5/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 15s 55ms/step - accuracy: 0.0137 - loss: 4.1312 - val_accuracy: 0.0176 - val_loss: 4.1290
Epoch 6/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 13s 48ms/step - accuracy: 0.0154 - loss: 4.1375 - val_accuracy: 0.0176 - val_loss: 4.1286
Epoch 7/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 13s 47ms/step - accuracy: 0.0189 - loss: 4.1383 - val_accuracy: 0.0176 - val_loss: 4.1284
Epoch 8/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 14s

# SO SÁNH VÀ PHÂN TÍCH

In [48]:
# 1. Bảng so sánh định lượng
results_df = pd.DataFrame({
    'Pipeline': [
        'TF-IDF + Logistic Regression',
        'Word2Vec (Avg) + Dense',
        'Embedding (Pre-trained) + LSTM',
        'Embedding (Scratch) + LSTM'
    ],
    'F1-score (Macro)': [
        f1_tfidf,
        f1_w2v,
        f1_lstm_pre,
        f1_lstm_scratch
    ],
    'Test Loss': [
        'N/A',
        f'{test_loss_w2v:.4f}',
        f'{test_loss_lstm_pre:.4f}',
        f'{test_loss_lstm_scratch:.4f}'
    ]
})

print("\nBẢNG SO SÁNH KẾT QUẢ:")
print(results_df.to_string(index=False))

# 2. Phân tích định tính
print("\n" + "=" * 60)
print("PHÂN TÍCH ĐỊNH TÍNH - CÁC CÂU KHÓ")
print("=" * 60)

# Chọn một số câu khó để test
test_sentences = [
    "can you remind me to not call my mom",
    "is it going to be sunny or rainy tomorrow", 
    "find a flight from new york to london but not through paris"
]

print("\nDự đoán trên các câu khó:")
for sent in test_sentences:
    print(f"\n📝 Câu: '{sent}'")
    
    # TF-IDF
    pred_tfidf = tfidf_lr_pipeline.predict([sent])[0]
    intent_tfidf = label_encoder.inverse_transform([pred_tfidf])[0]
    print(f"   TF-IDF + LR: {intent_tfidf}")
    
    # Word2Vec Average
    vec_avg = sentence_to_avg_vector(sent, w2v_model).reshape(1, -1)
    pred_w2v = np.argmax(w2v_model_dense.predict(vec_avg, verbose=0))
    intent_w2v = label_encoder.inverse_transform([pred_w2v])[0]
    print(f"   Word2Vec + Dense: {intent_w2v}")
    
    # LSTM Pre-trained
    seq = tokenizer.texts_to_sequences([sent])
    seq_pad = pad_sequences(seq, maxlen=max_len, padding='post')
    pred_lstm_pre = np.argmax(lstm_model_pretrained.predict(seq_pad, verbose=0))
    intent_lstm_pre = label_encoder.inverse_transform([pred_lstm_pre])[0]
    print(f"   LSTM (Pre-trained): {intent_lstm_pre}")
    
    # LSTM Scratch
    pred_lstm_scratch = np.argmax(lstm_model_scratch.predict(seq_pad, verbose=0))
    intent_lstm_scratch = label_encoder.inverse_transform([pred_lstm_scratch])[0]
    print(f"   LSTM (Scratch): {intent_lstm_scratch}")

NameError: name 'f1_lstm_pre' is not defined